<a href="https://colab.research.google.com/github/dylanwalker/MGSC496/blob/main/MGSC496_NHLScraper_extracting_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting data from Hockey Teams Data on [this site](http://www.scrapethissite.com/pages/forms/)



To get started building a scraper for this site containing NHL data, the first thing I do is install parsel and import the Selector class (so I can play with xpaths). 

Next, I use the `requests` library to get the source of the page, name it `doc` and then make a Selector object out of it using:

 ```python
 selector = Selector(doc)
 ```

In [ ]:
%%capture
!pip install parsel

In [ ]:
from parsel import Selector
import requests
res = requests.get('http://www.scrapethissite.com/pages/forms/')
doc = res.text
selector = Selector(doc)

Once, I've done that I inspect various element of the page in my browser and figure out which pieces of data I want to extract, and then figure out an xpath for each one. I print out the variables as I go, so I can see what I'm extracting from this first page and keep adding variables that capture the data items until I get something that looks like this: 

In [ ]:
teams = selector.xpath('//table[@class="table"]//tr[@class="team"]') # seems like this xpath identifies each row which contains info about the team
for team in teams: # for each row I look for all the data I want to extract for that row; many have simple class names, so they are relatively easy to get an xpath for then
  team_name = team.xpath('td[@class="name"]/text()').get().strip() # I added the .strip() because there was a newline character I wanted to get rid of.
  year = team.xpath('td[@class="year"]/text()').get().strip()
  wins = team.xpath('td[@class="wins"]/text()').get().strip()
  losses = team.xpath('td[@class="losses"]/text()').get().strip()
  ot_losses = team.xpath('td[@class="ot_losses"]/text()').get() # this one was tricky, because it is sometimes a None object, so I tested for that case below 
  if ot_losses is None:
    ot_losses = '' # I made it an empty string when there is no text
  ot_losses = ot_losses.strip() # otherwise, I process it like any other
  win_pct = team.xpath('td[contains(@class,"pct")]/text()').get().strip()
  goals_for = team.xpath('td[@class="gf"]/text()').get().strip()
  goals_against = team.xpath('td[@class="ga"]/text()').get().strip()
  diff = team.xpath('td[contains(@class,"diff")]/text()').get().strip()
  print(f"{team_name} {year} {wins} {losses} {win_pct} {goals_for} {goals_against} {diff}")
next_link = selector.xpath('//a[@aria-label="Next"]/@href').get() #Each page has a lot of links to further pages at the bottom, but also a convenient next link, which is perfect for scraping
print(next_link)

Boston Bruins 1990 44 24 0.55 299 264 35
Buffalo Sabres 1990 31 30 0.388 292 278 14
Calgary Flames 1990 46 26 0.575 344 263 81
Chicago Blackhawks 1990 49 23 0.613 284 211 73
Detroit Red Wings 1990 34 38 0.425 273 298 -25
Edmonton Oilers 1990 37 37 0.463 272 272 0
Hartford Whalers 1990 31 38 0.388 238 276 -38
Los Angeles Kings 1990 46 24 0.575 340 254 86
Minnesota North Stars 1990 27 39 0.338 256 266 -10
Montreal Canadiens 1990 39 30 0.487 273 249 24
New Jersey Devils 1990 32 33 0.4 272 264 8
New York Islanders 1990 25 45 0.312 223 290 -67
New York Rangers 1990 36 31 0.45 297 265 32
Philadelphia Flyers 1990 33 37 0.412 252 267 -15
Pittsburgh Penguins 1990 41 33 0.512 342 305 37
Quebec Nordiques 1990 16 50 0.2 236 354 -118
St. Louis Blues 1990 47 22 0.588 310 250 60
Toronto Maple Leafs 1990 23 46 0.287 241 318 -77
Vancouver Canucks 1990 28 43 0.35 243 315 -72
Washington Capitals 1990 37 36 0.463 258 258 0
Winnipeg Jets 1990 26 43 0.325 260 288 -28
Boston Bruins 1991 36 32 0.45 270 275 -5

Now that I know how to get everything I want, I can make a new scraper and put all this data extraction code into the spider.